In [1]:
import pandas as pd

In [2]:
df1=pd.read_csv('final.csv')
df1

,Unnamed: 0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,class_encoded
0,0,-596.27124,105.507520,-16.348253,34.213352,-7.040627,9.673249,15.672894,-18.594084,12.680883,-3.053727,-9.320203,5.608082,-5.410851,0
1,1,-359.78244,72.187680,27.768223,38.328430,5.198799,8.380541,10.607283,-13.184477,11.505762,-4.186356,-6.031159,2.438568,-6.354579,0
2,2,-424.57343,57.615242,12.128860,49.243443,-2.526363,11.702657,11.698957,-2.770733,-1.388484,-1.123289,-11.638446,14.478795,-10.840614,0
3,3,-524.17444,103.392075,-7.327183,56.391010,10.343275,4.731972,23.960910,-0.654973,6.595408,4.164212,-6.080491,7.985336,0.112556,0
4,4,-519.28940,101.250860,20.647596,29.956661,8.789474,8.442513,3.100049,5.523725,0.429982,5.303805,-0.014299,4.767339,0.963716,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,-473.95490,94.092310,20.465342,14.455287,0.151350,3.607172,-2.468565,-5.190783,0.989746,-2.069601,-10.654523,0.043760,2.364469,3
64,64,-569.43933,38.809494,15.469359,35.134655,-19.263390,-2.875730,-5.993804,-12.542236,-8.285859,-0.797593,-6.813976,-10.226942,-7.345919,3
65,65,-534.11633,45.754597,6.438214,24.977623,1.711067,9.408805,0.141316,-0.821431,-1.418567,3.832415,-2.813458,-1.262796,-4.410855,3
66,66,-552.32140,65.580140,21.206110,12.841641,-0.550191,12.242992,-0.532321,-1.793944,2.008286,-4.199604,-3.661440,-2.616800,-4.645094,3


In [3]:
df1=df1.drop(columns=['Unnamed: 0'])

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = df1.iloc[:, :-1] 
y = df1.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42  # For reproducibility
)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("Training set distribution:")
print(train_df[y.name].value_counts(normalize=True))

print("\nTest set distribution:")
print(test_df[y.name].value_counts(normalize=True))

Training set distribution:
class_encoded
1    0.259259
3    0.259259
2    0.240741
0    0.240741
Name: proportion, dtype: float64

Test set distribution:
class_encoded
0    0.285714
2    0.285714
3    0.214286
1    0.214286
Name: proportion, dtype: float64


In [5]:
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [6]:
import numpy as np
import pandas as pd  # Ensure you import pandas if you're using DataFrames
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import keras_tuner as kt  # Use keras_tuner instead of kerastuner
from keras.utils import to_categorical

# Assuming X_train, X_test, y_train, and y_test are already defined

# Check if they are DataFrames and convert to NumPy arrays if necessary
if isinstance(X_train, pd.DataFrame):
    X_train = X_train.to_numpy()  # Convert to NumPy array
if isinstance(X_test, pd.DataFrame):
    X_test = X_test.to_numpy()    # Convert to NumPy array

# One-hot encoding the labels
num_classes = 4
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Reshape the input data for GRU (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Shape: (54, 1, 14)
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))      # Shape: (14, 1, 14)

def build_model(hp):
    model = Sequential()
    num_layers = hp.Int('num_layers', 3, 6)  # Limit to 3 to 6 layers to reduce complexity

    # Adding GRU layers
    model.add(GRU(hp.Int('gru_units_0', 16, 64, step=16), 
                   return_sequences=num_layers > 1, 
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(BatchNormalization())

    for i in range(1, num_layers):
        model.add(GRU(hp.Int(f'gru_units_{i}', 16, 64, step=16), 
                       return_sequences=i < num_layers - 1))
        model.add(BatchNormalization())
        
        # Hyperparameter for dropout rate
        dropout_rate = hp.Float(f'dropout_rate_{i}', 0.2, 0.5, step=0.1)
        model.add(Dropout(dropout_rate))

    # Output layer for categorical classification
    model.add(Dense(num_classes, activation='softmax'))

    optimizer_options = ['adam', 'sgd', 'rmsprop']
    model.compile(loss='categorical_crossentropy', optimizer=hp.Choice('optimizer', optimizer_options), metrics=['accuracy'])
    
    return model

# Initializing the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    project_name='gru_tuning'
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Start the tuning process with a fixed batch size
batch_size = 8
tuner.search(X_train, y_train, 
             epochs=50,
             validation_data=(X_test, y_test),
             batch_size=batch_size,  # Fixed batch size
             callbacks=[early_stopping])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()

Trial 20 Complete [00h 00m 15s]
val_loss: 1.385648488998413

Best val_loss So Far: 1.3839061260223389
Total elapsed time: 00h 07m 17s
Results summary
Results in .\gru_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 09 summary
Hyperparameters:
num_layers: 3
gru_units_0: 64
gru_units_1: 16
dropout_rate_1: 0.4
gru_units_2: 48
dropout_rate_2: 0.2
optimizer: sgd
gru_units_3: 32
dropout_rate_3: 0.30000000000000004
gru_units_4: 32
dropout_rate_4: 0.4
gru_units_5: 48
dropout_rate_5: 0.4
Score: 1.3839061260223389

Trial 07 summary
Hyperparameters:
num_layers: 3
gru_units_0: 48
gru_units_1: 32
dropout_rate_1: 0.4
gru_units_2: 48
dropout_rate_2: 0.30000000000000004
optimizer: sgd
gru_units_3: 48
dropout_rate_3: 0.2
gru_units_4: 48
dropout_rate_4: 0.30000000000000004
gru_units_5: 32
dropout_rate_5: 0.2
Score: 1.3842127323150635

Trial 18 summary
Hyperparameters:
num_layers: 4
gru_units_0: 16
gru_units_1: 32
dropout_rate_1: 0.30000000000000004
gru_units_2: 32
dropou

In [7]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 64)             15168     
                                                                 
 batch_normalization (BatchN  (None, 1, 64)            256       
 ormalization)                                                   
                                                                 
 gru_1 (GRU)                 (None, 1, 16)             3936      
                                                                 
 batch_normalization_1 (Batc  (None, 1, 16)            64        
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 1, 16)             0         
                                                                 
 gru_2 (GRU)                 (None, 48)                9

In [8]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Loss: 1.3839
Test Accuracy: 0.2857


In [10]:
best_model.save('gru.h5')